In [3]:
import torch
import torch.nn as nn
import flwr as fl
import numpy as np

# Define Global Model with Updated Architecture
class GlobalModel(nn.Module):
    def __init__(self, input_size):
        super(GlobalModel, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_size, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.layers(x)


input_size = 16  
global_model = GlobalModel(input_size)

# Secure Aggregation: Wait until at least 2 clients are connected
class CustomFedAvg(fl.server.strategy.FedAvg):
    def __init__(self):
        super().__init__()
        self.connected_clients = 0

    def wait_for_clients(self):
        print("[Server] Waiting for at least 2 clients to connect...")
        while self.connected_clients < 2:
            pass  # Hold until at least 2 clients join
        print("[Server] Minimum clients connected. Starting aggregation...")

    def aggregate_fit(self, rnd, results, failures):
        self.connected_clients = len(results)  # Update connected clients count
        self.wait_for_clients()
        print(f"[Server] Securely aggregating model updates for round {rnd}...")
        aggregated_result = super().aggregate_fit(rnd, results, failures)
        
        # Log Aggregated Loss
        loss_values = [res.metrics["loss"] for _, res in results if "loss" in res.metrics]
        avg_loss = np.mean(loss_values) if loss_values else 0.0
        print(f"Round {rnd}: Aggregated Loss: {avg_loss:.4f}")
        return aggregated_result

    def aggregate_evaluate(self, rnd, results, failures):
        print(f"[Server] Securely aggregating evaluation results for round {rnd}...")
        aggregated_result = super().aggregate_evaluate(rnd, results, failures)
        
        # Compute and Log Validation Accuracy
        accuracies = [res.metrics['accuracy'] for _, res in results if "accuracy" in res.metrics]
        avg_accuracy = np.mean(accuracies) if accuracies else 0.0
        print(f"[Server] Validation Accuracy after Round {rnd}: {avg_accuracy:.4f}")
        return aggregated_result

# Start Flower Server
print("[Server] Starting Federated Learning with Secure Aggregation...")
fl.server.start_server(
    server_address="localhost:8080",
    config=fl.server.ServerConfig(num_rounds=20),
    strategy=CustomFedAvg()
)

# Perform Prediction after Training
print("[Server] Training completed. Running final predictions...")

def predict(model, X_test):
    model.eval()
    with torch.no_grad():
        predictions = model(torch.tensor(X_test, dtype=torch.float32))
    return predictions.numpy()

# Placeholder for test data (Replace with actual test dataset)
X_test = np.random.rand(10, input_size)  # Example test data
predictions = predict(global_model, X_test)
print("[Server] Final Predictions:", predictions)


	Instead, use the `flower-superlink` CLI command to start a SuperLink as shown below:

		$ flower-superlink --insecure

	To view usage and all available options, run:

		$ flower-superlink --help

	Using `start_server()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        


INFO :      Starting Flower server, config: num_rounds=20, no round_timeout
INFO :      Flower ECE: gRPC server running (20 rounds), SSL is disabled
INFO :      [INIT]
INFO :      Requesting initial parameters from one random client


[Server] Starting Federated Learning with Secure Aggregation...


INFO :      Received initial parameters from one random client
INFO :      Starting evaluation of initial global parameters
INFO :      Evaluation returned no results (`None`)
INFO :      
INFO :      [ROUND 1]
INFO :      configure_fit: strategy sampled 2 clients (out of 2)
INFO :      aggregate_fit: received 2 results and 0 failures
INFO :      configure_evaluate: strategy sampled 2 clients (out of 2)
INFO :      aggregate_evaluate: received 2 results and 0 failures
INFO :      
INFO :      [ROUND 2]
INFO :      configure_fit: strategy sampled 2 clients (out of 2)


[Server] Waiting for at least 2 clients to connect...
[Server] Minimum clients connected. Starting aggregation...
[Server] Securely aggregating model updates for round 1...
Round 1: Aggregated Loss: 4.2137
[Server] Securely aggregating evaluation results for round 1...
[Server] Validation Accuracy after Round 1: 0.8514


...

[Server] Waiting for at least 2 clients to connect...
[Server] Minimum clients connected. Starting aggregation...
[Server] Securely aggregating model updates for round 19...
Round 19: Aggregated Loss: 2.9969
[Server] Securely aggregating evaluation results for round 19...
[Server] Validation Accuracy after Round 19: 0.8821


INFO :      configure_fit: strategy sampled 2 clients (out of 2)
INFO :      aggregate_fit: received 2 results and 0 failures
INFO :      configure_evaluate: strategy sampled 2 clients (out of 2)
INFO :      aggregate_evaluate: received 2 results and 0 failures
INFO :      
INFO :      [SUMMARY]
INFO :      Run finished 20 round(s) in 121.31s
INFO :      	History (loss, distributed):
INFO :      		round 1: 0.0
INFO :      		round 2: 0.0
INFO :      		round 3: 0.0
INFO :      		round 4: 0.0
INFO :      		round 5: 0.0
INFO :      		round 6: 0.0
INFO :      		round 7: 0.0
INFO :      		round 8: 0.0
INFO :      		round 9: 0.0
INFO :      		round 10: 0.0
INFO :      		round 11: 0.0
INFO :      		round 12: 0.0
INFO :      		round 13: 0.0
INFO :      		round 14: 0.0


[Server] Waiting for at least 2 clients to connect...
[Server] Minimum clients connected. Starting aggregation...
[Server] Securely aggregating model updates for round 20...
Round 20: Aggregated Loss: 2.9620
[Server] Securely aggregating evaluation results for round 20...
[Server] Validation Accuracy after Round 20: 0.8844


INFO :      		round 15: 0.0
INFO :      		round 16: 0.0
INFO :      		round 17: 0.0
INFO :      		round 18: 0.0
INFO :      		round 19: 0.0
INFO :      		round 20: 0.0
INFO :      


[Server] Training completed. Running final predictions...
[Server] Final Predictions: [[0.4990117 ]
 [0.50065994]
 [0.49773714]
 [0.4981152 ]
 [0.49538413]
 [0.49995708]
 [0.49782512]
 [0.49886596]
 [0.4939555 ]
 [0.49578154]]
